## Socioeconomic data and TOC entitlements
* Entitlements assigned to census tracts
* Which census tracts (what income levels or median household income) have seen TOC entitlements?
* See if tract is composed of mostly TOC-eligible parcels
* Then look at Census characteristics of mostly TOC-eligible tracts vs not

In [1]:
import boto3
import geopandas as gpd
import intake
import numpy as np
import os
import pandas as pd
import pcts_census_utils
import utils

In [2]:
catalog = intake.open_catalog("../catalogs/*.yml")

s3 = boto3.client('s3')
bucket_name = 'city-planning-entitlements'

## Identify this dichotomy of tracts that are mostly TOC-eligible vs not

In [3]:
def toc_tracts_clean_and_aggregate():
    # Import data
    crosswalk_parcels_tracts = pd.read_parquet(f's3://{bucket_name}/data/crosswalk_parcels_tracts.parquet')

    s3.download_file(bucket_name, f'gis/intermediate/TOC_Parcels.parquet', '../gis/TOC_Parcels.parquet')
    toc_parcels = gpd.read_parquet('../gis/TOC_Parcels.parquet')
    os.remove('../gis/TOC_Parcels.parquet')

    df = pd.merge(crosswalk_parcels_tracts, 
         toc_parcels[toc_parcels.TOC_Tier > 0].drop(columns = 'TOC_Tier'), 
         on = 'AIN', how = 'left', validate = '1:1')
    
    # Get rid of duplicate AIN's
    df = df[df.num_AIN == 1]
    
    # Tag if the parcel counts as in TOC tier or not
    def in_tier(row):
        if row.TOC_Tier != 0:
            return 1
        elif row.TOC_Tier == 0:
            return 0

    df = df.assign(
        in_tier = df.apply(in_tier, axis=1)
    )

    # Aggregate by in_tier 
    df = (df[df.in_tier==1]
          .groupby(["GEOID", "pop", "parcel_tot", "in_tier"])
          .agg({"num_AIN": "sum",
               "parcelsqft":"sum"})
          .reset_index()
         )
    
    # Also, count the total of AIN within each tract
    total_AIN = (crosswalk_parcels_tracts[crosswalk_parcels_tracts.num_AIN == 1]
                    .groupby(['GEOID'])
                    .agg({'num_AIN':'sum'})
                    .rename(columns = {'num_AIN':'total_AIN'})
                    .reset_index()
                   )
    
    # Merge together 
    df2 = pd.merge(df, total_AIN, on = 'GEOID', how = 'left', validate = 'm:1')
    
    # Calculate the % of AIN that falls within TOC tiers and % of area within TOC tiers
    df2 = df2.assign(
        pct_AIN = df2.num_AIN / df2.total_AIN,
        pct_area = df2.parcelsqft / df2.parcel_tot,
    )
    
    return df2

In [4]:
df = toc_tracts_clean_and_aggregate()

In [5]:
df.head()

,GEOID,pop,parcel_tot,in_tier,num_AIN,parcelsqft,total_AIN,pct_AIN,pct_area
0,06037104320,6211,7.666438e+06,1,396,2.828009e+06,901,0.439512,0.368882
1,06037104401,3775,3.926756e+06,1,166,1.334841e+06,504,0.329365,0.339935
2,06037104404,3046,4.030609e+06,1,260,2.510833e+06,451,0.576497,0.622941
3,06037104610,3501,4.530658e+06,1,437,3.720807e+06,531,0.822976,0.821251
4,06037104620,3449,4.296846e+06,1,230,1.753480e+06,573,0.401396,0.408085


## Merge in Census data

In [6]:
census = pd.read_parquet(f's3://{bucket_name}/data/final/census_cleaned.parquet')

In [43]:
# Commute mode
def transform_census_percent(table_name, year, main_var, aggregate_me, aggregated_row_name, denom, numer):
    df = pcts_census_utils.grab_census_table(table_name, year, main_var)

    df2 = pcts_census_utils.aggregate_group(df, aggregate_me, name = aggregated_row_name)
    
    cols = [denom, numer]
    df3 = pcts_census_utils.make_wide(df2, cols)
    
    new_var = f"pct_{aggregated_row_name}"

    df3 = (df3.assign(
        new = df3[numer] / df3[denom],
        ).rename(columns = {'new': new_var})
    )
    
    return df3

In [35]:
commute_modes = ["workers_transit", "workers_walk", "workers_bike"]

commute = transform_census_percent("commute", 2018, "workers", commute_modes, 
                           "non_car", "workers_total", "non_car")

commute.head()

In [44]:
# Renter occupied
tenure_group = ["renter"]

tenure = transform_census_percent("tenure", 2018, "pop", tenure_group, 
                           "pop_renter", "pop_total", "pop_renter")

tenure.head()

,GEOID,pop_renter,pop_total,pct_pop_renter
0,06037101110,2199,4219,0.521214
1,06037101122,577,3234,0.178417
2,06037101210,5247,5987,0.876399
3,06037101220,2110,3497,0.603374
4,06037101300,353,4250,0.083059


In [41]:
census[(census.table=="tenure") & (census.year==2018)].main_var.value_counts()

pop    4692
Name: main_var, dtype: int64